In [64]:
import pandas as pd 
import sys
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow import keras 
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from keras.models import Sequential, Model 
from keras.optimizers import Adam 
from keras.utils import plot_model

In [65]:
class Discriminator:
    
    def __init__(self, width=28, height =28, channels=1, latent_size=100) -> None:
        self.latent_size = latent_size
        self.CAPACITY = width*height*channels
        self.SHAPE = (width,height,channels)
        self.OPTIMIZER = Adam(learning_rate=0.002, beta_2=8e-9)
        
        self.Discriminator = self.model()
        
        # Compile the model 
        self.Discriminator.compile(loss='binary_crossentropy',
                                   optimizer=self.OPTIMIZER,
                                   metrics=['accuracy'])
        # get the summary of the model 
        self.summary()
    def model(self):
        model = Sequential()
        model.add(Flatten(input_shape=self.SHAPE)) # faltten the input later of the size width*height*channel
        model.add(Dense(self.CAPACITY,input_shape=self.SHAPE, activation=LeakyReLU(alpha=0.2), name='Dense_layer_1')) # first dense layer 
        model.add(Dense(int(self.CAPACITY/2), activation=LeakyReLU(alpha=0.2),name='Dense_layer_2'))
        model.add(Dense(1, activation='sigmoid',name='output_layer')) # Output Dense Layer 
        return model 
    
    def summary(self):
        self.Discriminator.summary()
    
    def save_model(self):
        plot_model(
            self.Discriminator.model,
            to_file='/data/Disriminator.png',
            show_shapes=True
        )

Discriminator(width=28,height=28,channels=1,latent_size=100)

Model: "sequential_84"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_35 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_layer_1 (Dense)           │ (None, 784)            │       615,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_layer_2 (Dense)           │ (None, 392)            │       307,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │           393 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 923,553 (3.52 MB)

 Trainable params: 923,553 (3.52 MB)

 Non-trainable params: 0 (0.00 B)

In [66]:
from tensorflow import keras 
from keras.layers import BatchNormalization, Dense, Reshape, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

# Generator class 
class Generator:
    
    def __init__(self,width=28,height=28,channels=1, latent_size = 100) -> None:
        self.width = width 
        self.height = height
        self.channels = channels
        self.CAPACITY = width*height*channels
        self.SHAPE = (width,height,channels)
        self.LATENT_SPACE_SIZE= latent_size
        
        self.OPTIMIZER = Adam(learning_rate=0.002, beta_2=8e-9)
        
        # Generate the Random array of the Laten_space_size 
        self.latent_space = np.random.normal(0,1,(self.LATENT_SPACE_SIZE,))

        self.Generator = self.model()
        self.Generator.compile(loss='binary_crossentropy',
                               optimizer = self.OPTIMIZER)
        self.summary()
        
    def model(self,block_starting_size=128,num_blocks=4):
        block_size  = block_starting_size
        model = Sequential()
        model.add(Dense(block_size , input_shape=(self.LATENT_SPACE_SIZE,),activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        for i in range(1,num_blocks-1) :
            # block_size = block_size*2*i
            block_size = block_size*2
            model.add(Dense(block_size,activation=LeakyReLU(0.2),))
            model.add(BatchNormalization(momentum=0.8))
        
        # Output Layer 
        model.add(Dense(self.CAPACITY,activation='tanh'))
        model.add(Reshape(self.SHAPE))
        
        return model
    def summary(self):
        self.Generator.summary()
    
    def save_model(self):
        plot_model(
            self.Generator.model,
            to_file='/data/Generator.png',
            show_shapes=True
        )
Generator()

Model: "sequential_85"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_116 (Dense)               │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_85          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_86          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_87          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 784)            │       402,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_27 (Reshape)            │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 583,312 (2.23 MB)

 Trainable params: 581,520 (2.22 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [67]:
import sys
import numpy as np
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import plot_model

class GAN(object):
    def __init__(self,discriminator,generator):
        self.OPTIMIZER = Adam(learning_rate=0.0002, beta_2=8e-9)
        
        self.Generator = generator

        self.Discriminator = discriminator
        self.Discriminator.trainable = False
        
        self.gan_model = self.model()
        self.gan_model.compile(loss='binary_crossentropy', optimizer=self.OPTIMIZER)
        self.summary()
        # self.save_model()

    def model(self):
        model = Sequential()
        model.add(self.Generator)
        model.add(self.Discriminator)
        return model

    def summary(self):
        return self.gan_model.summary()

    def save_model(self):
        plot_model(self.gan_model.model, to_file='/data/GAN_Model.png')

GAN(Discriminator().Discriminator,Generator().Generator)

Model: "sequential_86"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_36 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_layer_1 (Dense)           │ (None, 784)            │       615,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_layer_2 (Dense)           │ (None, 392)            │       307,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │           393 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 923,553 (3.52 MB)

 Trainable params: 923,553 (3.52 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "sequential_88"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_87 (Sequential)      │ ?                      │       583,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_86 (Sequential)      │ ?                      │       923,553 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,506,865 (5.75 MB)

 Trainable params: 581,520 (2.22 MB)

 Non-trainable params: 925,345 (3.53 MB)

In [68]:
# Trainig the GAN model 
from keras.datasets import mnist 
from random import randint
import numpy as np 
import matplotlib.pyplot as plt 

class Trainer:
    
    def __init__(self,
                 width=28,height=28,channels=1,
                 latent_size = 100, epochs=50000, batch=32,
                 checkpoint = 5000, model_type =-1) -> None:
        self.width = width 
        self.height = height 
        self.channels = channels
        self.EPOCHS = epochs 
        self.BATCH = batch 
        self.model_type = model_type
        self.LATENT_SPACE_SIZE = latent_size
        self.CHECKPOINT = checkpoint
        
        self.generator = Generator(width=self.width,height=self.height,channels=self.channels)
        self.discriminator = Discriminator(width=self.width,height=self.height,channels=self.channels)
        self.gan = GAN(discriminator=self.discriminator.Discriminator, generator=self.generator.Generator)
        
        self.load_MNIST()
        
    def load_MNIST(self,model_type=3):
        allowed_types = [-1,0,1,2,3,4,5,6,7,8,9]
        if self.model_type not in allowed_types:
            print('ERROR: Only Integer Values from -1 to 9 are allowed')
        
        (self.X_train, self.Y_train), (_, _) = mnist.load_data()
        if self.model_type!=-1:
            self.X_train =self.X_train[np.where(self.Y_train==int(self.model_type)[0])]
        
        self.X_train = ( np.float32(self.X_train) - 127.5) / 127.5
        self.X_train = np.expand_dims(self.X_train, axis=3)
        
        return 
    
    
    def train(self):
        
        for e in range(self.EPOCHS):
            # Trian the Discriminator 
            # MAke the Training Batch for thiis model be half of the real, half of the noise 
            # Grab Real images for this trainig 
            
            count_real_images = int(self.BATCH/2)
            starting_index = randint(0,(len(self.X_train)-count_real_images))
            real_images_raw = self.X_train[ starting_index : (starting_index + count_real_images) ]
            x_real_images = real_images_raw.reshape( count_real_images, self.width, self.height, self.channels )
            y_real_labels = np.ones([count_real_images,1])
            # print(f'Shape of the X real images {x_real_images.shape}')
            # Grab Generated Images for this training batch
            latent_space_samples = self.sample_latent_space(count_real_images)
            x_generated_images = self.generator.Generator.predict(latent_space_samples)
            y_generated_labels = np.zeros([self.BATCH-count_real_images,1])
            # print(f'Shape of the X generated images {x_generated_images.shape}')
            
            # Combine to train on the discriminator
            x_batch = np.concatenate( [x_real_images, x_generated_images] )
            y_batch = np.concatenate( [y_real_labels, y_generated_labels] )
            
            # Now, train the discriminator with this batch
            discriminator_loss = self.discriminator.Discriminator.train_on_batch(x_batch,y_batch)[0]
            
                        # Generate Noise
            x_latent_space_samples = self.sample_latent_space(self.BATCH)
            # print(f'Shape of the x_latent_space_samples {x_latent_space_samples.shape}')
            y_generated_labels = np.ones([self.BATCH,1])
            generator_loss = self.gan.gan_model.train_on_batch(x_latent_space_samples,y_generated_labels)

            # print ('Epoch: '+str(int(e))+', [Discriminator :: Loss: '+str(discriminator_loss)+'], [ Generator :: Loss: '+str(generator_loss)+']')
                        
            if e % self.CHECKPOINT == 0 :
                self.plot_checkpoint(e)
                
    def sample_latent_space(self, instances):
        return np.random.normal(0, 1, (instances,self.LATENT_SPACE_SIZE))
    
    def plot_checkpoint(self,e):
        filename = "data/sample_"+str(e)+".png"

        noise = self.sample_latent_space(16)
        images = self.generator.Generator.predict(noise)
        
        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.height,self.width])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close('all')
        return
            

In [69]:
HEIGHT  = 28
WIDTH   = 28
CHANNEL = 1
LATENT_SPACE_SIZE = 100
EPOCHS = 50001
BATCH = 32
CHECKPOINT = 50
MODEL_TYPE = -1

trainer = Trainer(height=HEIGHT,\
                 width=WIDTH,\
                 channels=CHANNEL,\
                 latent_size=LATENT_SPACE_SIZE,\
                 epochs =EPOCHS,\
                 batch=BATCH,\
                 checkpoint=CHECKPOINT,
                 model_type=MODEL_TYPE)
                 
trainer.train()

Model: "sequential_89"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_124 (Dense)               │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_91          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_125 (Dense)               │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_92          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_126 (Dense)               │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_93          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_127 (Dense)               │ (None, 784)            │       402,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_29 (Reshape)            │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 583,312 (2.23 MB)

 Trainable params: 581,520 (2.22 MB)

 Non-trainable params: 1,792 (7.00 KB)

...

 Total params: 1,506,865 (5.75 MB)

 Trainable params: 581,520 (2.22 MB)

 Non-trainable params: 925,345 (3.53 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 